In [10]:
!pip install nvidia-pyindex
!pip install nvidia-dllogger

  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8399 sha256=7ce2ce5aa71dcf5c5c3975151372debdcde8631f0d6c46692e820150ba4afe00
  Stored in directory: /root/.cache/pip/wheels/e0/c2/fb/5cf4e1cfaf28007238362cb746fb38fc2dd76348331a748d54
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
# Run the pre-processing script to calculate pitch:
!(python prepare_dataset.py --wav-text-filelists dataset/tts_data.txt --n-workers 0 --batch-size 1 --dataset-path dataset --extract-pitch --f0-method pyin --extract-mels)

DLL 2022-12-05 10:51:58.352150 - PARAMETER dataset_path : dataset 
DLL 2022-12-05 10:51:58.352266 - PARAMETER wav_text_filelists : ['dataset/tts_data.txt'] 
DLL 2022-12-05 10:51:58.352331 - PARAMETER extract_mels : True 
DLL 2022-12-05 10:51:58.352394 - PARAMETER extract_pitch : True 
DLL 2022-12-05 10:51:58.352443 - PARAMETER save_alignment_priors : False 
DLL 2022-12-05 10:51:58.352491 - PARAMETER log_file : preproc_log.json 
DLL 2022-12-05 10:51:58.352538 - PARAMETER n_speakers : 1 
DLL 2022-12-05 10:51:58.352585 - PARAMETER max_wav_value : 32768.0 
DLL 2022-12-05 10:51:58.352633 - PARAMETER sampling_rate : 22050 
DLL 2022-12-05 10:51:58.352679 - PARAMETER filter_length : 1024 
DLL 2022-12-05 10:51:58.352725 - PARAMETER hop_length : 256 
DLL 2022-12-05 10:51:58.352772 - PARAMETER win_length : 1024 
DLL 2022-12-05 10:51:58.352818 - PARAMETER mel_fmin : 0.0 
DLL 2022-12-05 10:51:58.352874 - PARAMETER mel_fmax : 8000.0 
DLL 2022-12-05 10:51:58.352931 - PARAMETER n_mel_channels : 80 
DL

In [5]:
# Prepare file lists with paths to pre-calculated pitch:
def create_picth_text_file():
    manifest_path = 'dataset/tts_data.txt'
    with open(manifest_path, 'r', encoding="utf-8") as f:  
        Lines = f.readlines()
        for line in Lines:
            new_line = line.strip()[0:line.find('|')] + "|pitch" + \
        line.strip()[line.find('/'):line.find('.wav')] + ".pt" + line.strip()[line.find('|'):]
            with open('dataset/tts_pitch_data.txt', 'a', encoding="utf-8") as fn:        
                fn.write(new_line)
                fn.write('\n')
                
create_picth_text_file()

In [18]:
!(python train.py --cuda --p-arpabet 1.0 --dataset-path dataset --output /saved_fastpicth_models --training-files dataset/tts_pitch_data.txt --validation-files dataset/tts_pitch_data.txt --epochs 100 --learning-rate 0.001 --batch-size 8 --load-pitch-from-disk)

DLL 2022-12-07 07:03:01.638766 - PARAMETER | output :  /saved_fastpicth_models
DLL 2022-12-07 07:03:01.638963 - PARAMETER | dataset_path :  dataset
DLL 2022-12-07 07:03:01.639052 - PARAMETER | log_file : 
DLL 2022-12-07 07:03:01.639128 - PARAMETER | epochs :  100
DLL 2022-12-07 07:03:01.639213 - PARAMETER | epochs_per_checkpoint :  5
DLL 2022-12-07 07:03:01.639305 - PARAMETER | checkpoint_path : 
DLL 2022-12-07 07:03:01.639393 - PARAMETER | keep_milestones :  [100, 200, 300, 400, 500, 600, 700, 800, 900]
DLL 2022-12-07 07:03:01.639560 - PARAMETER | resume :  False
DLL 2022-12-07 07:03:01.639638 - PARAMETER | seed :  1234
DLL 2022-12-07 07:03:01.639712 - PARAMETER | amp :  False
DLL 2022-12-07 07:03:01.639789 - PARAMETER | cuda :  True
DLL 2022-12-07 07:03:01.639882 - PARAMETER | cudnn_benchmark :  False
DLL 2022-12-07 07:03:01.639990 - PARAMETER | ema_decay :  0
DLL 2022-12-07 07:03:01.640063 - PARAMETER | grad_accumulation :  1
DLL 2022-12-07 07:03:01.640174 - PARAMETER | kl_loss_star